在这个教程中，你将会学到怎么用Folium制作可交互地图可视化  

提供的基础数据是：
<div class="alert alert-info"><h2>提供的基础数据是：</h2><p>   
    GIS文件：<br>  
    1.深圳行政区划<br>  
    2.深圳栅格<br>  
    <br>  
    数据：<br>  
    1.出租车GPS集计栅格OD<br>  

 </p></div>

In [ ]:
#导入必要的包
import pandas as pd
import numpy as np

#绘制图用的包
import matplotlib as mpl
import matplotlib.pyplot as plt

#geopandas包
import geopandas

#shapely包
from shapely.geometry import Point,Polygon,shape

# 读取基础数据

## GIS数据

In [ ]:
#行政区划数据
#读取shapefile文件
shp = r'shapefile\sz.shp'
xzqh = geopandas.GeoDataFrame.from_file(shp,encoding = 'utf-8')

#绘制看看长什么样
xzqh.plot()

In [ ]:
#栅格数据
#读取shapefile文件
shp =  r'shapefile\grid\grid.shp'
grid = geopandas.GeoDataFrame.from_file(shp,encoding = 'gbk')

#绘制看看长什么样
grid.plot()

## 栅格OD数据

这个数据是前面教程2中，用公式计算出来的OD

In [ ]:
import pandas as pd
OD = pd.read_csv(r'data-sample\taxi_od_grid.csv')
OD.head(5)

# Folium可视化工具教程

  folium是js上著名的地理信息可视化库leaflet.js为Python提供的接口，通过它，我们可以通过在Python端编写代码操纵数据，来调用leaflet的相关功能，基于内建的osm或自行获取的osm资源和地图原件进行地理信息内容的可视化，以及制作优美的可交互地图。其语法格式类似ggplot2，是通过不断添加图层元素来定义一个Map对象，最后以几种方式将Map对象展现出来。
    
  而在Map对象的生成形式上，可以在定义所有的图层内容之后，将其保存为html文件在浏览器中独立显示，也可以基于jupyter notebook在一个ipynb文件内部嵌入对应的交互地图，本文即采用后者对应的方法。

In [ ]:
import folium

好的结束  

不，没有结束，pip install folium  
我安装好了

In [ ]:
import folium

## 栅格绘制

首先，创建地图

In [ ]:
#快看一下创建地图有哪些参数
?folium.Map

In [ ]:
###############################你需要在下面写代码##################################
# 在深圳行政区划的中心xzqh创建地图
# 调整缩放水平为10

###################################################################################

m

In [ ]:
###############################     答  案      ##################################

'''
其主要参数如下：
location：tuple或list类型输入，用于控制初始地图中心点的坐标，格式为(纬度，经度)或[纬度，经度]，默认为None
width：int型或str型，int型时，传入的是地图宽度的像素值；str型时，传入的是地图宽度的百分比，形式为'xx%'。默认为'100%'
height：控制地图的高度，格式同width
tiles：str型，用于控制绘图调用的地图样式，默认为'OpenStreetMap'，也有一些其他的内建地图样式，如'Stamen  Terrain'、'Stamen Toner'、'Mapbox Bright'、'Mapbox Control Room'等；也可以传入'None'来绘制一个没有风格的朴素地图，或传入一个URL来使用其它的自选osm
max_zoom：int型，控制地图可以放大程度的上限，默认为18
attr：str型，当在tiles中使用自选URL内的osm时使用，用于给自选osm命名
control_scale：bool型，控制是否在地图上添加比例尺，默认为False即不添加
no_touch：bool型，控制地图是否禁止接受来自设备的触控事件譬如拖拽等，默认为False，即不禁止
'''
m = folium.Map(location=[xzqh.unary_union.centroid.y,xzqh.unary_union.centroid.x],
              zoom_start=10)
###################################################################################
m

然后，在地图上添加东西

folium的图形绘制支撑geojson格式，这种格式可以理解为一种以文本格式存储的地理空间数据，类似shp，只不过是以另一个标准保存  
从geopandas很容易可以得到geojson格式，对geodataframe用.to_json()方法就行

In [ ]:
folium.GeoJson(
    xzqh.to_json(),
    name='深圳行政区划'
).add_to(m)
m

In [ ]:
###############################你需要在下面写代码##################################
#我们对OD数据的['SLONCOL','SLATCOL']字段集计，也就是对O点集计
#然后再连接到grid上，命名为gridtoplot


###################################################################################

gridtoplot.head(5)

In [ ]:
###############################     答  案      ##################################
#我们对OD数据的['SLONCOL','SLATCOL']字段集计，也就是对O点集计
#然后再连接到grid上，命名为gridtoplot
O = OD.groupby(['SLONCOL','SLATCOL'])['VehicleNum'].sum().reset_index()
O.columns = ['LONCOL','LATCOL','count']
gridtoplot =pd.merge(grid,O,on = ['LONCOL','LATCOL'])

###################################################################################

In [ ]:
#这里我们只显示count为10个以上的格子，否则矢量图象太多显示不出来
gridtoplot = gridtoplot[gridtoplot['count']>10]

In [ ]:
#好的我已经写好了这个函数
def foliumgridplot(m,grid,column,fill_color='YlOrRd',fill_opacity = 0.7,line_opacity =0.1,legend_name = '',name = ''):
    folium.Choropleth(
        name = name,
        geo_data=grid.reset_index().to_json(),
        data=grid[column].reset_index(),
        columns=['index', column],
        key_on='feature.properties.index',
        fill_color=fill_color,
        fill_opacity=fill_opacity,
        line_opacity=line_opacity,
        legend_name=legend_name
    ).add_to(m)
    return m

#直接这样就能绘制
foliumgridplot(m,gridtoplot,'count',name = '出租车O点分布')

In [ ]:
#给右边添加一个图层管理器
folium.LayerControl().add_to(m)

m

In [ ]:
#给罗湖火车站打个标签
folium.Marker([22.530967,114.113088],
              popup='罗湖火车站',
              icon=folium.Icon(icon='fa-train',prefix = 'fa')).add_to(m)
m

[这个网页上有各种图标](https://faicons.com/)

## 热力图

In [ ]:
gridtoplot[['HBLAT','HBLON','count']].head(5)

注：先纬度后经度

In [ ]:
?HeatMap

In [ ]:
from folium.plugins import HeatMap
###############################你需要在下面写代码##################################
#在图上加入热力图


###################################################################################


m

In [ ]:
from folium.plugins import HeatMap
###############################     答  案      ##################################
#在图上加入热力图
m = folium.Map(location=[xzqh.unary_union.centroid.y,xzqh.unary_union.centroid.x],
              zoom_start=11)

HeatMap(gridtoplot[['HBLAT','HBLON','count']].values,max_val=1,radius=13,blur = 25).add_to(m)


###################################################################################


m

Folium的优点很明显，出GIS图需要代码很少，与geopandas的兼容性很好  
不过用Folium可视化出图，亲测一万个矢量图形就已经很卡了  
所以大规模（矢量元素达到万级别）批量出（一次性出多张图）专题图还是用geopandas+matplotlib吧  
  
更多案例请参考：[宇宙最大同性交友网站](https://github.com/python-visualization/folium)

# 作业

尝试用Folium绘制OD图